<a href="https://colab.research.google.com/github/JYHYL/EART60702-Group3/blob/Stacking/1-data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import os

**Methodology**

Since both Random Forest and LSTM models utilize a sliding window approach for prediction, the effective lengths of the training and testing predictions are shorter than the original input periods. Therefore, the period from April 1, 2006 to December 31, 2040 is defined as the new training set, and April 1, 2050 to November 30, 2080 as the new testing set. Corresponding data are extracted from the LSTM_prediction and RF_prediction results.


As Model 85 is based on monthly-scale data, its prediction outputs are linearly interpolated to match the daily resolution. The new datasets are stored in the DataSet directory in Excel format with the following file names: stacking_data_model_train_<model_id> and stacking_data_model_test_<model_id>.


Each dataset includes the following features：

a. LSTM predictions based on the original model (LSTM_day)

b. RF predictions based on the original model (RF_day)

c. LSTM predictions based on Model 85 (LSTM_85)

d. RF predictions based on Model 85 (RF_85)

Extracting Training Data for Each Model

In [8]:
# ---------------- Configuration ----------------
day_models = [3, 4, 5, 6, 7, 8]         # Daily models (MME)
month_model = 85                        # Monthly model (CHESS)
start_date = '2006-04-01'
end_date = '2040-12-31'

# GitHub raw base URLs (different branches for LSTM and RF)
base_url_lstm = 'https://raw.githubusercontent.com/JYHYL/EART60702-Group3/LSTM_prediction'
base_url_rf = 'https://raw.githubusercontent.com/JYHYL/EART60702-Group3/RF'

# Directory structure in GitHub (relative to branches)
lstm_dir_day = 'LSTM_MME_prediction'
rf_dir_day = 'RF_MME/RF_MME_prediction'
lstm_dir_month = 'LSTM_CHESS_prediction'
rf_dir_month = 'RF_CHESS/RF_CHESS_predictions'

# ---------------- Step 1: Collect common dates shared by all day models ----------------
date_sets = []

for model_id in day_models:
    file_url = f'{base_url_lstm}/{lstm_dir_day}/model_{model_id}_train_predictions.csv'
    df = pd.read_csv(file_url, parse_dates=['date'])
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    date_sets.append(set(df['date']))

# Intersect all date sets to get common prediction dates
common_dates = sorted(set.intersection(*date_sets))
df_full_dates = pd.DataFrame({'date': common_dates})

# ---------------- Step 2: Interpolate monthly LSTM predictions from Model 85 ----------------
file_lstm_85 = f'{base_url_lstm}/{lstm_dir_month}/model_85_train_predictions.csv'
df_lstm_85_raw = pd.read_csv(file_lstm_85, parse_dates=['date'])
df_lstm_85_raw = df_lstm_85_raw[(df_lstm_85_raw['date'] >= start_date) & (df_lstm_85_raw['date'] <= end_date)]

df_lstm_85_interp = df_lstm_85_raw.set_index('date').reindex(df_full_dates['date'])
df_lstm_85_interp['Predicted'] = df_lstm_85_interp['Predicted'].interpolate(method='linear', limit_direction='both')
df_lstm_85_interp = df_lstm_85_interp.reset_index().rename(columns={'index': 'date', 'Predicted': 'LSTM_85'})

# ---------------- Step 3: Interpolate monthly RF predictions from Model 85 ----------------
file_rf_85 = f'{base_url_rf}/{rf_dir_month}/model_85_train_predictions.csv'
df_rf_85_raw = pd.read_csv(file_rf_85, parse_dates=['date'])
df_rf_85_raw = df_rf_85_raw[(df_rf_85_raw['date'] >= start_date) & (df_rf_85_raw['date'] <= end_date)]

df_rf_85_interp = df_rf_85_raw.set_index('date').reindex(df_full_dates['date'])
df_rf_85_interp['Predicted'] = df_rf_85_interp['Predicted'].interpolate(method='linear', limit_direction='both')
df_rf_85_interp = df_rf_85_interp.reset_index().rename(columns={'index': 'date', 'Predicted': 'RF_85'})

# ---------------- Step 4: Create and save stacking datasets ----------------
for model_id in day_models:
    # Load LSTM predictions for the current model
    file_lstm = f'{base_url_lstm}/{lstm_dir_day}/model_{model_id}_train_predictions.csv'
    df_lstm = pd.read_csv(file_lstm, parse_dates=['date'])
    df_lstm = df_lstm[(df_lstm['date'] >= start_date) & (df_lstm['date'] <= end_date)]
    df_lstm = df_lstm[['date', 'Predicted']].rename(columns={'Predicted': 'LSTM_day'})

    # Load RF predictions for the current model
    file_rf = f'{base_url_rf}/{rf_dir_day}/model_{model_id}_train_predictions.csv'
    df_rf = pd.read_csv(file_rf, parse_dates=['date'])
    df_rf = df_rf[(df_rf['date'] >= start_date) & (df_rf['date'] <= end_date)]
    df_rf = df_rf[['date', 'Predicted']].rename(columns={'Predicted': 'RF_day'})

    # Load actual values (reusing LSTM file)
    df_actual = pd.read_csv(file_lstm, parse_dates=['date'])
    df_actual = df_actual[(df_actual['date'] >= start_date) & (df_actual['date'] <= end_date)]
    df_actual = df_actual[['date', 'Actual']]

    # Merge all features into a single DataFrame
    df_merge = df_full_dates.merge(df_lstm, on='date', how='left')
    df_merge = df_merge.merge(df_rf, on='date', how='left')
    df_merge = df_merge.merge(df_lstm_85_interp, on='date', how='left')
    df_merge = df_merge.merge(df_rf_85_interp, on='date', how='left')
    df_merge = df_merge.merge(df_actual, on='date', how='left')

    # Reorder columns
    df_merge = df_merge[['date', 'LSTM_day', 'RF_day', 'LSTM_85', 'RF_85', 'Actual']]

    # Save to Excel file
    output_name = f'stacking_data_model_train_{model_id}.xlsx'
    df_merge.to_excel(output_name, index=False)


Extracting Testing Data for Each Model

In [9]:
# ---------------- Configuration ----------------
day_models = [3, 4, 5, 6, 7, 8]
month_model = 85
start_date = '2050-04-01'
end_date = '2080-11-30'

# GitHub raw base URLs for each branch
base_url_lstm = 'https://raw.githubusercontent.com/JYHYL/EART60702-Group3/LSTM_prediction'
base_url_rf = 'https://raw.githubusercontent.com/JYHYL/EART60702-Group3/RF'

lstm_dir_day = 'LSTM_MME_prediction'
rf_dir_day = 'RF_MME/RF_MME_prediction'
lstm_dir_month = 'LSTM_CHESS_prediction'
rf_dir_month = 'RF_CHESS/RF_CHESS_predictions'

# ---------------- Step 1: Collect common dates shared across all models ----------------
date_sets = []

for model_id in day_models:
    file_url = f'{base_url_lstm}/{lstm_dir_day}/model_{model_id}_test_predictions.csv'
    df = pd.read_csv(file_url, parse_dates=['date'])
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    date_sets.append(set(df['date']))

common_dates = sorted(set.intersection(*date_sets))
df_full_dates = pd.DataFrame({'date': common_dates})

# ---------------- Step 2: Interpolate monthly LSTM predictions (Model 85) ----------------
file_lstm_85 = f'{base_url_lstm}/{lstm_dir_month}/model_85_test_predictions.csv'
df_lstm_85_raw = pd.read_csv(file_lstm_85, parse_dates=['date'])
df_lstm_85_raw = df_lstm_85_raw[(df_lstm_85_raw['date'] >= start_date) & (df_lstm_85_raw['date'] <= end_date)]

df_lstm_85_interp = df_lstm_85_raw.set_index('date').reindex(df_full_dates['date'])
df_lstm_85_interp['Predicted'] = df_lstm_85_interp['Predicted'].interpolate(method='linear', limit_direction='both')
df_lstm_85_interp = df_lstm_85_interp.reset_index().rename(columns={'index': 'date', 'Predicted': 'LSTM_85'})

# ---------------- Step 3: Interpolate monthly RF predictions (Model 85) ----------------
file_rf_85 = f'{base_url_rf}/{rf_dir_month}/model_85_test_predictions.csv'
df_rf_85_raw = pd.read_csv(file_rf_85, parse_dates=['date'])
df_rf_85_raw = df_rf_85_raw[(df_rf_85_raw['date'] >= start_date) & (df_rf_85_raw['date'] <= end_date)]

df_rf_85_interp = df_rf_85_raw.set_index('date').reindex(df_full_dates['date'])
df_rf_85_interp['Predicted'] = df_rf_85_interp['Predicted'].interpolate(method='linear', limit_direction='both')
df_rf_85_interp = df_rf_85_interp.reset_index().rename(columns={'index': 'date', 'Predicted': 'RF_85'})

# ---------------- Step 4: Build and save stacking test datasets ----------------
for model_id in day_models:
    # Load LSTM predictions for the current model
    file_lstm = f'{base_url_lstm}/{lstm_dir_day}/model_{model_id}_test_predictions.csv'
    df_lstm = pd.read_csv(file_lstm, parse_dates=['date'])
    df_lstm = df_lstm[(df_lstm['date'] >= start_date) & (df_lstm['date'] <= end_date)]
    df_lstm = df_lstm[['date', 'Predicted']].rename(columns={'Predicted': 'LSTM_day'})

    # Load RF predictions for the current model
    file_rf = f'{base_url_rf}/{rf_dir_day}/model_{model_id}_test_predictions.csv'
    df_rf = pd.read_csv(file_rf, parse_dates=['date'])
    df_rf = df_rf[(df_rf['date'] >= start_date) & (df_rf['date'] <= end_date)]
    df_rf = df_rf[['date', 'Predicted']].rename(columns={'Predicted': 'RF_day'})

    # Load actual values (from LSTM prediction file)
    df_actual = pd.read_csv(file_lstm, parse_dates=['date'])
    df_actual = df_actual[(df_actual['date'] >= start_date) & (df_actual['date'] <= end_date)]
    df_actual = df_actual[['date', 'Actual']]

    # Merge all columns by date
    df_merge = df_full_dates.merge(df_lstm, on='date', how='left')
    df_merge = df_merge.merge(df_rf, on='date', how='left')
    df_merge = df_merge.merge(df_lstm_85_interp, on='date', how='left')
    df_merge = df_merge.merge(df_rf_85_interp, on='date', how='left')
    df_merge = df_merge.merge(df_actual, on='date', how='left')

    # Reorder columns
    df_merge = df_merge[['date', 'LSTM_day', 'RF_day', 'LSTM_85', 'RF_85', 'Actual']]

    # Save to Excel
    output_name = f'stacking_data_model_test_{model_id}.xlsx'
    df_merge.to_excel(output_name, index=False)
